In [1]:
from langchain_community.document_loaders.directory import DirectoryLoader
from langchain_community.document_loaders.json_loader import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.vectorstores.elasticsearch import ElasticsearchStore
from langchain_core.runnables import RunnableMap

from elasticsearch import Elasticsearch

from tqdm import tqdm
# import os
# import json
# from bs4 import BeautifulSoup
# from elqm.utils import get_data_dir

In [2]:
# data_dir = get_data_dir("elqm-raw/eur_lex_data")
# print(data_dir)

In [3]:
# preprocesses_dir = get_data_dir("elqm-preprocessed/eur_lex_data")
# print(preprocesses_dir)

In [4]:
# for filename in os.listdir(data_dir):
#     if filename.endswith(".json"):
#         with open(os.path.join(data_dir, filename), 'r') as f:
#             data = json.load(f)
        
#         bs = BeautifulSoup(data['html'], 'html.parser')

#         # Get the text
#         text = bs.get_text()

#         data['text'] = text
#         del data['html']

#         with open(os.path.join(preprocesses_dir, filename), 'w') as f:
#             json.dump(data, f)

In [5]:
# loader = DirectoryLoader(preprocesses_dir,
#     glob='**/*.json',
#     show_progress=True,
#     loader_cls=JSONLoader,
#     loader_kwargs={'jq_schema': '.text'}
# )

# data = loader.load()

In [6]:
# splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# chunks = splitter.split_documents(data)

# len(chunks)

In [7]:
embeddings = GPT4AllEmbeddings()

# vectorstore = FAISS.from_documents(documents=chunks, embedding=embeddings)
# vectorstore = ElasticsearchStore.from_documents(
#     documents=chunks,
#     embedding=embeddings,
#     index_name="eurlex-langchain-troubleshoot",
#     es_connection=Elasticsearch("http://localhost:9200")
# ) 
vectorstore = ElasticsearchStore(
    index_name="eurlex-langchain-troubleshoot",
    es_connection=Elasticsearch("http://localhost:9200"),
    embedding=embeddings)

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [8]:
question = "Can the product's heat output be regulated by an energy supplier? "

In [9]:
retriever_chain = vectorstore.as_retriever()

In [10]:
retrieved_docs = retriever_chain.invoke(question)
len(retrieved_docs)

4

In [11]:
retriever_chain_formatted = retriever_chain

In [12]:
# This works
for i in tqdm(range(100)):
    retriever_chain_formatted.invoke(input=question)

100%|██████████| 100/100 [00:00<00:00, 215.98it/s]


In [13]:
all_chain = RunnableMap(runnables={
    "retriever": retriever_chain,
    "retriever_formatted": retriever_chain_formatted,
})

In [14]:
import time

In [15]:
# This does not work
for i in tqdm(range(100)):
    results_retriever_chain = retriever_chain.invoke(input=question)
    results_retriever_chain_formatted = retriever_chain_formatted.invoke(input=question)

100%|██████████| 100/100 [00:00<00:00, 102.27it/s]


In [ ]:
# # This does not work
# for i in tqdm(range(100)):
#     results = all_chain.invoke(input=question)
#     time.sleep(0.5)

  4%|▍         | 4/100 [00:02<00:48,  1.98it/s]

: 